# Landing Layer to Raw Layer (renaming columns, data type conversions, etc.)

In [1]:
from pyspark.sql import SparkSession



# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/06 15:07:52 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/08/06 15:07:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/06 15:07:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/06 15:07:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Yellow Taxis

In [1]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/tlc'):
    os.makedirs('../data/raw/tlc')
    
if not os.path.exists('../data/raw/tlc/yellow'):
    os.makedirs('../data/raw/tlc/yellow')

### Check data types

In [4]:
yellow_landing_dir = '../data/landing/tlc/yellow/'

sdf_yellow_2022_05 = spark.read.parquet(f"{yellow_landing_dir}2022-05.parquet")

sdf_yellow_all = spark.read.parquet(f"{yellow_landing_dir}*")


In [5]:
sdf_yellow_2022_05.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [42]:
sdf_yellow_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

### Schemas appear to be incorrect for all of 2022, so we will adjust some data types and ensure consistent lowercase casing

In [6]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# convert data types

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'VendorID', 
    F.col('VendorID').cast('Integer') 
)

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'PULocationID', 
    F.col('PULocationID').cast('Integer') 
)

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'DOLocationID', 
    F.col('DOLocationID').cast('Integer') 
)

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'payment_type', 
    F.col('payment_type').cast('Integer') 
)

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'RatecodeID', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_yellow_2022_05 = sdf_yellow_2022_05.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_yellow_2022_05.schema



StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [7]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_yellow_2022_05.columns]
sdf_yellow_2022_05 = sdf_yellow_2022_05.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_yellow_2022_05.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [8]:
# convert all landing datasets to required schema

yellow_landing_dir = '../data/landing/tlc/yellow/'
    
yellow_raw_dir = '../data/raw/tlc/yellow/'

YEARS = ['2022']
MONTHS = range(5,12)

for year in YEARS:
    for month in MONTHS:

        month = str(month).zfill(2) 

        sdf_malformed = spark.read.parquet(f"{yellow_landing_dir}{year}-{month}.parquet")

        # select all columns from the existing malformed dataframe and cast it to the required schema and change casing, drop ehail_fee as it has only NULL values
        sdf_malformed = sdf_malformed \
            .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
            .select(*consistent_col_casing) \
        
        sdf_malformed \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet(f"{yellow_raw_dir}{year}-{month}.parquet")



In [9]:
# double check schema of raw layer data

sdf1 = spark.read.parquet('../data/raw/tlc/yellow/*')

sdf1.show(1, vertical=True, truncate=100)

#sdf1.show(1, vertical=True, truncate=100)

''' checkinglanding vs raw dataset shape and landing dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 tpep_pickup_datetime  | 2022-05-01 00:00:36 
 tpep_dropoff_datetime | 2022-05-01 00:19:18 
 passenger_count       | 1                   
 trip_distance         | 4.1                 
 ratecodeid            | 1                   
 store_and_fwd_flag    | N                   
 pulocationid          | 246                 
 dolocationid          | 151                 
 payment_type          | 2                   
 fare_amount           | 17.0                
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 20.8                
 congestion_surcharge  | 2.5                 
 airport_fee           | 0.0                 
only showing top 1 row



" checkinglanding vs raw dataset shape and landing dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

## Taxi Zones

In [19]:
# make directories for raw layer

import os
import geopandas as gpd

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/tlc'):
    os.makedirs('../data/raw/tlc')
    
if not os.path.exists('../data/raw/tlc/taxi_zones'):
    os.makedirs('../data/raw/tlc/taxi_zones')

In [21]:
taxi_zone_lookup = spark.read.option("header",True) \
                   .csv("../data/landing/tlc/taxi_zones/taxi+_zone_lookup.csv")

In [10]:
print(taxi_zone_lookup.limit(5))

print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])


### Just need to convert the column names to lowercase for this data

In [23]:
from pyspark.sql import functions as F

# converting to lower case
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in taxi_zone_lookup.columns]
taxi_zone_lookup = taxi_zone_lookup.select(*consistent_col_casing)



print(taxi_zone_lookup.limit(5))
print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('locationid', StringType(), True), StructField('borough', StringType(), True), StructField('zone', StringType(), True), StructField('service_zone', StringType(), True)])


In [24]:
# save raw layer data as parquet
taxi_zone_lookup \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet')
    


In [25]:
# check raw layer data
taxi_zone_lookup = spark.read.parquet('../data/raw/tlc/taxi_zones/taxi_zone_lookup.parquet')

taxi_zone_lookup.printSchema


<bound method DataFrame.printSchema of +----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City

In [26]:
# move taxi_zones.zip to raw layer data
import shutil

src = '../data/landing/tlc/taxi_zones/taxi_zones.zip'

dst = '../data/raw/tlc/taxi_zones/taxi_zones.zip'

shutil.copyfile(src, dst)

import zipfile

# unzip shapefile
with zipfile.ZipFile(dst, 'r') as zip_ref:
    zip_ref.extractall('../data/raw/tlc/taxi_zones')

## External Datasets

### Subway Entrances and Exits

In [2]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/subway_entr_exit'):
    os.makedirs('../data/raw/subway_entr_exit')


In [3]:
subway_entr_exit = spark.read.parquet('../data/landing/subway_entr_exit/subway_entr_exit.parquet')

subway_entr_exit.show(5)

+--------+----------------+--------------------+
|division|            line|    station_location|
+--------+----------------+--------------------+
|     IRT|        Flushing|(40.755882, -74.0...|
|     IND|        8 Avenue|(40.740893, -74.0...|
|     BMT|        Canarsie|(40.739777, -74.0...|
|     IRT|Broadway-7th Ave|(40.737826, -74.0...|
|     IRT|Broadway-7th Ave|(40.733422, -74.0...|
+--------+----------------+--------------------+
only showing top 5 rows



#### Remove division and line

In [4]:
subway_entr_exit = subway_entr_exit.select('station_location')

In [5]:
subway_entr_exit \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/subway_entr_exit/subway_entr_exit.parquet')

### Hotel Properties

In [6]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/hotels'):
    os.makedirs('../data/raw/hotels')


In [7]:
hotels = spark.read.parquet('../data/landing/hotels/hotels.parquet')

hotels.schema

hotels.show(5)

+----------+--------+-----+----+-------+-------------+----------------+--------+----------+--------+--------------------+---------+---------+----------+---------------+----------------+------------+---------+-------------+--------------------+
|     PARID|BOROCODE|BLOCK| LOT|TAXYEAR|STREET NUMBER|     STREET NAME|Postcode|BLDG_CLASS|TAXCLASS|          OWNER_NAME|  Borough| Latitude| Longitude|Community Board|Council District|Census Tract|      BIN|          BBL|                 NTA|
+----------+--------+-----+----+-------+-------------+----------------+--------+----------+--------+--------------------+---------+---------+----------+---------------+----------------+------------+---------+-------------+--------------------+
|1000080039|       1|    8|  39|   2021|           32|    PEARL STREET|   10004|        H3|       4|       32 PEARL, LLC|MANHATTAN|40.703235|-74.012421|          101.0|             1.0|         9.0|1078968.0|1.000080039E9|Battery Park City...|
|1000080051|       1|   

#### Convert to lowercase, remove useless features

In [8]:
hotels = hotels.select('TAXYEAR', 'Borough', 'Latitude', 'Longitude')

# consistent lowercasing and underscores where there are spaces
consistent_col_casing = [F.col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in hotels.columns]
hotels = hotels.select(*consistent_col_casing)

hotels.show(5)

+-------+---------+---------+----------+
|taxyear|  borough| latitude| longitude|
+-------+---------+---------+----------+
|   2021|MANHATTAN|40.703235|-74.012421|
|   2021|MANHATTAN|40.702744|-74.012201|
|   2021|MANHATTAN|40.704025|-74.012638|
|   2021|MANHATTAN|40.704039|-74.012317|
|   2021|MANHATTAN|40.714812|-74.016153|
+-------+---------+---------+----------+
only showing top 5 rows



In [9]:
hotels \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/hotels/hotels.parquet')


### Airbnb

In [7]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/airbnb'):
    os.makedirs('../data/raw/airbnb')


In [17]:
airbnb = spark.read.parquet('../data/landing/airbnb/airbnb.parquet')

airbnb.schema

airbnb.show(5)

+----+--------------------+-------+---------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------+
|  id|                name|host_id|host_name|neighbourhood_group|     neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|number_of_reviews_ltm|license|
+----+--------------------+-------+---------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+---------------------+-------+
|5136|Spacious Brooklyn...|   7378|  Rebecca|           Brooklyn|       Sunset Park|40.66265|-73.99454|Entire home/apt|  275|            21|                3| 2022-08-10|             0.03|           

#### Convert to lowercase, remove useless features

In [18]:
airbnb = airbnb.select('neighbourhood_group', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'reviews_per_month')

# consistent lowercasing and underscores where there are spaces
consistent_col_casing = [F.col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in airbnb.columns]
airbnb = airbnb.select(*consistent_col_casing)

airbnb.show(5)

airbnb = airbnb.withColumnRenamed(
    'price',
    'daily_price_usd'    
)

+-------------------+--------+---------+---------------+-----+--------------+-----------------+
|neighbourhood_group|latitude|longitude|      room_type|price|minimum_nights|reviews_per_month|
+-------------------+--------+---------+---------------+-----+--------------+-----------------+
|           Brooklyn|40.66265|-73.99454|Entire home/apt|  275|            21|             0.03|
|          Manhattan| 40.8038|-73.96751|   Private room|   75|             2|             0.73|
|           Brooklyn|40.68535|-73.95512|   Private room|   60|            30|              0.3|
|          Manhattan|40.76457|-73.98317|   Private room|   68|             2|             3.38|
|          Manhattan|40.75356|-73.98559|Entire home/apt|  175|            30|             0.31|
+-------------------+--------+---------+---------------+-----+--------------+-----------------+
only showing top 5 rows



In [19]:
airbnb \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/airbnb/airbnb.parquet')


### Census

In [24]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/census'):
    os.makedirs('../data/raw/census')


In [28]:
census_tracts = spark.read.parquet('../data/landing/census/census_tracts.parquet')

census_tracts.schema

census_tracts.show(5)

+-----------+------+-------+--------+----+-----+--------+-----+-----+------+-----+-------+-------+---------+------------+---------------+-------+------------+------------+-------+------+------------+----------+-----+-------+-------+----+-----------+----------+-----------+--------+-----------+----------+------------+----------+------------+
|CensusTract|County|Borough|TotalPop| Men|Women|Hispanic|White|Black|Native|Asian|Citizen| Income|IncomeErr|IncomePerCap|IncomePerCapErr|Poverty|ChildPoverty|Professional|Service|Office|Construction|Production|Drive|Carpool|Transit|Walk|OtherTransp|WorkAtHome|MeanCommute|Employed|PrivateWork|PublicWork|SelfEmployed|FamilyWork|Unemployment|
+-----------+------+-------+--------+----+-----+--------+-----+-----+------+-----+-------+-------+---------+------------+---------------+-------+------------+------------+-------+------+------------+----------+-----+-------+-------+----+-----------+----------+-----------+--------+-----------+----------+------------

In [29]:
census_block_loc = spark.read.parquet('../data/landing/census/census_block_loc.parquet')

census_block_loc.schema

census_block_loc.show(5)

+--------+--------------+---------------+---------+-----+
|Latitude|     Longitude|      BlockCode|   County|State|
+--------+--------------+---------------+---------+-----+
|   40.48|        -74.28|340230076002012|Middlesex|   NJ|
|   40.48|-74.2768341709|340230076005000|Middlesex|   NJ|
|   40.48|-74.2736683417|340230076003018|Middlesex|   NJ|
|   40.48|-74.2705025126|340230076003004|Middlesex|   NJ|
|   40.48|-74.2673366834|340230074021000|Middlesex|   NJ|
+--------+--------------+---------------+---------+-----+
only showing top 5 rows



#### Convert to lowercase, remove useless features

In [30]:
census_tracts = census_tracts.drop('County', 'income',  'incomeerr', 'incomepercaperr', 'childpoverty', 'men', 'women', 'hispanic', 'black', 'white', 'native', 'asian', 'citizen', 'employed', 'othertransp', 'familywork')

# consistent lowercasing and underscores where there are spaces
consistent_col_casing = [F.col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in census_tracts.columns]
census_tracts = census_tracts.select(*consistent_col_casing)

census_tracts.show(5)

+-----------+-------+--------+------------+-------+------------+-------+------+------------+----------+-----+-------+-------+----+----------+-----------+-----------+----------+------------+------------+
|censustract|borough|totalpop|incomepercap|poverty|professional|service|office|construction|production|drive|carpool|transit|walk|workathome|meancommute|privatework|publicwork|selfemployed|unemployment|
+-----------+-------+--------+------------+-------+------------+-------+------+------------+----------+-----+-------+-------+----+----------+-----------+-----------+----------+------------+------------+
|36005000100|  Bronx|    7703|      2440.0|   null|        null|   null|  null|        null|      null| null|   null|   null|null|      null|       null|       null|      null|        null|        null|
|36005000200|  Bronx|    5403|     22180.0|   20.0|        28.7|   17.1|  23.9|         8.0|      22.3| 44.8|   13.7|   38.6| 2.9|       0.0|       43.0|       80.8|      16.2|         2.9

In [31]:
census_block_loc = census_block_loc.select('latitude', 'longitude', 'blockcode')

# consistent lowercasing and underscores where there are spaces
consistent_col_casing = [F.col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in census_block_loc.columns]
census_block_loc = census_block_loc.select(*consistent_col_casing)

census_block_loc.show(5)

+--------+--------------+---------------+
|latitude|     longitude|      blockcode|
+--------+--------------+---------------+
|   40.48|        -74.28|340230076002012|
|   40.48|-74.2768341709|340230076005000|
|   40.48|-74.2736683417|340230076003018|
|   40.48|-74.2705025126|340230076003004|
|   40.48|-74.2673366834|340230074021000|
+--------+--------------+---------------+
only showing top 5 rows



In [32]:
census_tracts \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/census/census_tracts.parquet')
        
census_block_loc \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/census/census_block_loc.parquet')


### Parking Munimeters

In [49]:
# make directories for raw layer

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/raw/'):
    os.makedirs('../data/raw/')
    
if not os.path.exists('../data/raw/parking'):
    os.makedirs('../data/raw/parking')


In [21]:
parking = spark.read.parquet('../data/landing/parking/parking.parquet')

parking.schema

parking.show(5)

+--------+------------+------+------------------+--------------------+---------------------+---------+---------+-----------------+--------------+--------------------+----------------+----------------+-----------------+----------------+----------------+--------------------+
|ObjectID|Meter Number|Status|Pay By Cell Number|         Meter_Hours|Parking_Facility_Name| Facility|  Borough|        On_Street|Side_of_Street|         From_Street|       To_Street|        Latitude|        Longitude|               X|               Y|            Location|
+--------+------------+------+------------------+--------------------+---------------------+---------+---------+-----------------+--------------+--------------------+----------------+----------------+-----------------+----------------+----------------+--------------------+
|   11750|     1083076|Active|            113091|2HR Pas Mon-Sat 0...|                 null|On Street|Manhattan|   West 85 Street|             N|    Amsterdam Avenue|        Broa

#### Convert to lowercase, remove useless features

In [22]:
parking = parking.select('status', 'borough', 'latitude', 'longitude')

# consistent lowercasing and underscores where there are spaces
consistent_col_casing = [F.col(col_name).alias(col_name.lower().replace(" ", "_")) for col_name in parking.columns]
parking = parking.select(*consistent_col_casing)

parking.show(5)

+------+---------+----------------+-----------------+
|status|  borough|        latitude|        longitude|
+------+---------+----------------+-----------------+
|Active|Manhattan|40.7875051180999|-73.9765950135923|
|Active|Manhattan|40.7461997160805|-73.9869404709221|
|Active|Manhattan|40.7784865293823|-73.9564275260737|
|Active|   Queens|40.7354393855834|-73.8148380161902|
|Active|Manhattan|40.7399045161829|-73.9825104553208|
+------+---------+----------------+-----------------+
only showing top 5 rows



In [23]:
parking \
        .coalesce(1) \
        .write \
        .mode('overwrite') \
        .parquet('../data/raw/parking/parking.parquet')
